In [79]:
import pandas as pd
import re
from sadie.airr.airrtable import LinkedAirrTable
from sadie.reference import Reference
from sadie.reference.yaml import YamlRef
from Levenshtein import distance
from itertools import combinations

# Read in and pre-process data

This is all 10X VDJ contigs and sorted cells in one dataframe.

In [80]:
# read in combined dataframe
all_combined = pd.read_feather("all_processed_combined.feather")

In [81]:
# convert to to a LinkedAirrTable (a heavy-light paired airrtable)
lat = LinkedAirrTable(all_combined)

## Add Closest Orthologs and CDR3 Length

here we are taking the macaque gene and finding the closest human ortholog

In [82]:
def get_lookups(human_lookup, mac_lookup):
    '''A simple function to find the best scoring ortholog between human and macaque'''
    lookups = {}
    for x in mac_lookup:
        lowest_score = 1000
        for y in human_lookup:
            d = distance(human_lookup[y], mac_lookup[x])
            if d < lowest_score:
                lookups[x] = y
                lowest_score = d
    return lookups

# enter no file to use reference.yml
yml_ref = YamlRef()

# create empty reference object
ref_class = Reference()
genes = yml_ref["human"]["imgt"]["human"]
ref_class.add_genes(**{"genes": genes, "species": "human", "source": "imgt"})

# make human
human_df = ref_class.get_dataframe()
v_genes_human = human_df.query("gene_segment=='V'")
d_genes_human = human_df.query("gene_segment=='D'")
j_genes_human = human_df.query("gene_segment=='J'")


# create empty reference object
ref_class = Reference()
genes = yml_ref["macaque"]["custom"]["macaque"]
ref_class.add_genes(**{"genes": genes, "species": "macaque", "source": "custom"})

# make macaque ref
macaque_df = ref_class.get_dataframe()
v_genes_macaque = macaque_df.query("gene_segment=='V'")
d_genes_macaque = macaque_df.query("gene_segment=='D'")
j_genes_macaque = macaque_df.query("gene_segment=='J'")

# now make a lookup table for macaque and human genes. 
mac_lookup_v = (
    v_genes_macaque.query("gene_segment=='V'")
    .query("`imgt.contrived_functional`=='F'")[["gene", "imgt.sequence_gapped_aa"]]
    .rename({"imgt.sequence_gapped_aa": "sequence_aa"}, axis=1)
    .set_index("gene")["sequence_aa"]
    .to_dict()
)

human_lookup_v = (
    v_genes_human.query("gene_segment=='V'")
    .query("`imgt.contrived_functional`=='F'")[["gene", "imgt.sequence_gapped_aa"]]
    .rename({"imgt.sequence_gapped_aa": "sequence_aa"}, axis=1)
    .set_index("gene")["sequence_aa"]
    .to_dict()
)

# do the same for the J gene
mac_lookup_j = (
    j_genes_macaque.query("gene_segment=='J'")[["gene", "imgt.sequence_gapped_aa"]]
    .rename({"imgt.sequence_gapped_aa": "sequence_aa"}, axis=1)
    .set_index("gene")["sequence_aa"]
    .to_dict()
)

human_lookup_j = (
    j_genes_human[["gene", "imgt.sequence_gapped_aa"]]
    .rename({"imgt.sequence_gapped_aa": "sequence_aa"}, axis=1)
    .set_index("gene")["sequence_aa"]
    .to_dict()
)

# for each human v, find the closest mac V based on the tables
closest_ortholog_v = get_lookups(human_lookup_v, mac_lookup_v)
closest_ortholog_j = get_lookups(human_lookup_j, mac_lookup_j)

In [83]:
# put in linked airrtable the orthologs 
lat["v_ortholog_heavy"] = lat["v_call_top_heavy"].map(closest_ortholog_v)
lat["v_ortholog_light"] = lat["v_call_top_light"].map(closest_ortholog_v)
lat["j_ortholog"] = lat["j_call_top_heavy"].map(closest_ortholog_j)

# where it's NA, put the word NONE
lat.loc[lat[lat["v_ortholog_light"].isna()].index, "v_ortholog_light"] = "NONE"

In [84]:
# add HCDR3 Length
lat["hcdr3_len"] = lat["cdr3_aa_heavy"].str.len()
lat["lcdr3_len"] = lat["cdr3_aa_light"].str.len()

# for any NA on CDR3 aa, just put length as 0. These will fall out on cluster
lat.loc[lat[lat["cdr3_aa_heavy"].isna()].index, "hcdr3_len"] = 0
lat.loc[lat[lat["cdr3_aa_light"].isna()].index, "lcdr3_len"] = 0

# also put the family in the dataframe
lat["v_ortholog_heavy_family"] = lat["v_ortholog_heavy"].str.split("-").str.get(0)
lat["v_ortholog_light_family"] = lat["v_ortholog_light"].str.split("-").str.get(0)

# assert we have no NA
assert not lat["v_ortholog_heavy_family"].isna().any()
assert not lat["v_ortholog_light_family"].isna().any()
assert not lat["hcdr3_len"].isna().any()
assert not lat["lcdr3_len"].isna().any()

# BG18 Type I Definitions

For all HCDR3 >= 22, find `ITIFG[LV]VI[IT]` and up to 4 mutations from  `ITIFG[LV]VI[IT]`

In [85]:
def split_multi_regex(seq):
    '''This groups the regex into segments and put them into a list
    
    example: split_multi_regex(split_multi_regex("ITIFG[LV]VI[IT]")
    
    >>>['I', 'T', 'I', 'F', 'G', '[LV]', 'V', 'I', '[IT]']
    
    '''
    new_seq = []
    inside = False
    inside_seq = ""
    for i in seq:

        if i == "[":
            inside = True
            inside_seq += "["
            continue
        if i == "]":
            new_seq.append(inside_seq + "]")
            inside = False
            inside_seq = ""
            continue
        if inside:
            inside_seq += i
        else:
            new_seq.append(i)
    return new_seq


def get_regex_combinations(regex, tolerance):
    '''Get all regex combinations
    
    ex: get_regex_combinations("ITIFG[LV]VI[IT]",4)
    
     'ITI.G[LV]VI[IT]',
     'ITIF.[LV]VI[IT]',
     'ITIFG.VI[IT]',
     'ITIFG[LV].I[IT]',
     'ITIFG[LV]V.[IT]',
     'ITIFG[LV]VI.',
     'ITIFG[LV]VI[IT]',
     'ITIFG[LV]VI[IT]'
     ...
    
    '''
    def mutate_base(positons):
        base_seq = split_multi_regex(regex)
        for x in positons:
            base_seq[x] = "."
        return "".join(base_seq)

    regexes = []
    for x in range(tolerance, -1, -1):
        for y in list(combinations(range(len(split_multi_regex(regex))), x)):
            regexes.append(mutate_base(list(y)))
    regexes.append(regex)
    return regexes


def get_best_standard_def(df, column, regex, tolerance):
    '''For every row in dataframe, find the best regex and best regex + '.E' and report it as a list of dictionaries
    
    ex:
    >>>best_regex, best_regex_e = get_best_standard_def(
        lat, "cdr3_aa_heavy", "ITIFG[LV]VI[IT]", 9
    )
    
    >>>best_regex
     {'index': 45608, 'best_regex': 'ITIFG[LV]VI.', 'tolerated_d_mutations': 1},
     {'index': 45607, 'best_regex': 'ITIFG[LV]VI.', 'tolerated_d_mutations': 1},
     {'index': 45606, 'best_regex': 'ITIFG[LV]VI.', 'tolerated_d_mutations': 1},
    >>>best_regex_e
     {'index': 34124,'best_regex_e': 'ITIFG[LV]VI[IT].E','tolerated_d_mutations': 0}
    '''
    matching_indexes = []
    matching_indexes_w_e = []
    regexes = get_regex_combinations(regex, tolerance)
    for regex in regexes:
        index = df[df[column].str.contains(regex, na=False)].index
        e_regex = regex + ".E"
        e_index = df[df[column].str.contains(e_regex, na=False)].index
        for i in index:
            matching_indexes.append(
                {
                    "index": i,
                    "best_regex": regex,
                    "tolerated_d_mutations": regex.count("."),
                }
            )
        for i in e_index:
            matching_indexes_w_e.append(
                {
                    "index": i,
                    "best_regex_e": e_regex,
                    "tolerated_d_mutations": regex.count("."),
                }
            )
    return matching_indexes, matching_indexes_w_e


def find_regex_pos(X):
    '''If you find a regex, find where it starts'''
    r = re.search(str(X[1]), str(X[0]))
    if r:
        return r.start() + 1

# find best regex and tolerated d mutations
best_regex, best_regex_e = get_best_standard_def(
    lat, "cdr3_aa_heavy", "ITIFG[LV]VI[IT]", 9
)


# best regex to df and get best tolerated d mutations
best_regexs = (
    pd.DataFrame(best_regex)
    .sort_values("tolerated_d_mutations")
    .groupby("index")
    .head(1)
)
# best regex_e to df and get best tolerated d mutations
best_e_regexes = (
    pd.DataFrame(best_regex_e)
    .sort_values("tolerated_d_mutations")
    .groupby("index")
    .head(1)
)


# add regexes to LinkedAirrTable
all_combined_w_def = lat.join(best_regexs.set_index("index"), how="left")

# add regexes with E to LinkedAirrTable
all_combined_w_def = all_combined_w_def.join(
    best_e_regexes.rename(
        {"tolerated_d_mutations": "tolerated_d_mutations_w_e"}, axis=1
    ).set_index("index"),
    how="left",
)

# find out where the best_regex_starts
all_combined_w_def["best_regex_start"] = all_combined_w_def[
    ["cdr3_aa_heavy", "best_regex"]
].apply(find_regex_pos, axis=1)